In [99]:
import illustris_python.groupcat as gc
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import numpy as np
import abacus_cosmos.Halos as ach
from astropy.table import Table
import h5py


%matplotlib inline

In [83]:
BoxID = 10
basePath = "/Users/forero/github/abacus/data/AbacusCosmos_720box_{:02d}_FoF_halos_z0.100/".format(BoxID)
halo_data = ach.read_halos_FoF(basePath)

In [84]:
print(halo_data['pos'].min())

-360.0


In [70]:
hubble = 0.67
BoxSize = 720.0

In [85]:
halo_data['pos'] = halo_data['pos']+BoxSize/2.0


0.0


In [90]:
print(halo_data[:10])
print(Table(halo_data[:1]))

[(33242377525485,     0, 5731, 58166, [44379,   434,   302,   263], [ 67.85376 , 703.6271  , 255.09146 ], [ 509.94177  ,  295.99063  , -272.3032   ], [1061.0593 ,  805.7593 ,  731.78394], 0.48171747, 0.80373114, 1.1528836 , 1.7118766, 1417.7762, 1.1274769 , [-292.11053  ,  343.66425  , -104.909706 ], [ 508.18326 ,  280.0467  , -251.75488 ], [1061.2086 ,  805.9302 ,  731.84357], 0.47740415, 0.80419564, 1.1546255 , 1.6933568, 1418.7654, 1.1052033 )
 (33242377525486,  5731, 4692, 46664, [26401,  1705,   830,   337], [153.27827 , 716.8886  , 353.4101  ], [-269.46985  ,  453.1271   ,  271.3301   ], [ 880.7662 ,  740.1681 ,  728.64343], 0.38679054, 0.719261  , 1.3305287 , 1.9388833, 1321.1968, 0.6265267 , [-206.70566  ,  356.97067  ,   -6.4816904], [-297.32925 ,  330.8448  ,  183.07785 ], [ 889.2765 ,  745.46484,  729.0449 ], 0.36268893, 0.7067062 , 1.3516446 , 1.9914364, 1330.4701, 0.48325482)
 (33242377525487, 10423, 4585, 45562, [31489,  1019,   212,   179], [ 25.486359, 707.0409  ,  70.9

In [91]:
print(np.min(halo_data['vcirc_max']))

54.944195


In [92]:
ii = halo_data['vcirc_max']>200 # in units of km/s
print(np.count_nonzero(ii))

912451


In [95]:
S_pos = halo_data['pos'][ii]
S_vel = halo_data['vel'][ii]
S_vmax = halo_data['vcirc_max'][ii]
S_parent_fof = halo_data['id'][ii]
n_S = len(S_pos)
print(n_S)

912451


In [76]:
#pad boxes around the S3 positions to mimic periodic boundary conditions
S_pad_pos = S_pos.copy()
S_pad_vel = S_vel.copy()
S_pad_vmax = S_vmax.copy()
S_pad_stellar_mass = S_stellar_mass.copy()
S_pad_fof = S_parent_fof.copy()
S_pad_id = np.arange(n_S)
for i in [0]:
    for j in [0]:
        for k in [0]:
            new_pos = S_pos.copy()
            if(i):
                new_pos[:,0] = new_pos[:,0] + i*BoxSize
            if(j):
                new_pos[:,1] = new_pos[:,1] + j*BoxSize
            if(k):
                new_pos[:,2] = new_pos[:,2] + k*BoxSize
                
            if((i!=0) | (j!=0) | (k!=0)):
                S_pad_pos = np.append(S_pad_pos, new_pos, axis=0)
                S_pad_vel = np.append(S_pad_vel, S_vel, axis=0)
                S_pad_vmax = np.append(S_pad_vmax, S_vmax)
                S_pad_stellar_mass = np.append(S_pad_stellar_mass, S_stellar_mass)
                S_pad_id = np.append(S_pad_id, np.arange(n_S))
                S_pad_fof = np.append(S_pad_fof, S_parent_fof)

In [77]:
nbrs_S = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(S_pad_pos)
dist_S, ind_S = nbrs_S.kneighbors(S_pad_pos)
print(S_pad_pos.shape)
print(dist_S.shape)

(912451, 3)
(912451, 20)


In [78]:
neighbor_index = ind_S[:,1]
neighbor_list = ind_S[:,2:]
print(np.shape(neighbor_list))

n_pairs = 0

halo_A_id = np.empty((0), dtype=int)
halo_B_id = np.empty((0), dtype=int)

for i in range(n_S):
#for i in range(10):
    l = neighbor_index[neighbor_index[i]]% n_S
    j = neighbor_index[i] % n_S
    
    other_j = neighbor_list[i,:] % n_S
    other_l = neighbor_list[neighbor_index[i],:] % n_S
    
    if((i==l) & (not (j in halo_A_id)) & (not (i in halo_B_id))): # first check to find mutual neighbors
        if((dist_S[i,1] > 0.7)): #check on the distance between the two galaxies
            #print('distancia')
            vmax_i = S_pad_vmax[i]
            vmax_j = S_pad_vmax[j]
            vmax_limit = min([vmax_i, vmax_j])
                
            pair_d = dist_S[i,1] # This is the current pair distance
            dist_limit = pair_d * 3.0 # exclusion radius for massive structures
            
            massive_close_to_i = any((dist_S[i,2:]<dist_limit) & (S_pad_vmax[other_j] >= vmax_limit))
            massive_close_to_j = any((dist_S[j,2:]<dist_limit) & (S_pad_vmax[other_l] >= vmax_limit))
            if((not massive_close_to_i) & (not massive_close_to_j)): # check on massive structures inside exclusion radius
                n_pairs = n_pairs+ 1
                halo_A_id = np.append(halo_A_id, int(i))
                halo_B_id = np.append(halo_B_id, int(j))
                    #print(pair_d)
print(n_pairs)

(912451, 18)
37853


In [108]:
filename = '../data/summary_box_{:02d}.hdf5'.format(BoxID)
h5f = h5py.File(filename, 'w')
h5f.create_dataset('pos_A', data=S_pos[halo_A_id,:])
h5f.create_dataset('pos_B', data=S_pos[halo_B_id,:])
h5f.create_dataset('pos_G', data=S_pos[ll,:])
h5f.create_dataset('vel_A', data=S_vel[halo_A_id,:])
h5f.create_dataset('vel_B', data=S_vel[halo_B_id,:])
h5f.create_dataset('vel_G', data=S_vel[ll,:])
h5f.create_dataset('vmax_A', data=S_vmax[halo_A_id])
h5f.create_dataset('vmax_B', data=S_vmax[halo_B_id])
h5f.create_dataset('vmax_G', data=S_vmax[ll])
h5f.close()